# Resource constrained project scheduling
This is a port of the Python-MIP example at https://python-mip.readthedocs.io/en/latest/examples.html#resource-constrained-project-scheduling

In [ ]:
%matplotlib widget
import processscheduler as ps

# Input
Compared to Python-MIP, there's no need to create dummy jobs

In [ ]:
# total number of jobs
n = 10

# job durations
p = [3, 2, 5, 4, 2, 3, 4, 2, 4, 6]

# number of resources [c1, c2] required per for each job
u = [[5, 1], [0, 4], [1, 4], [1, 3], [3, 2], [3, 1], [2, 4], [4, 0], [5, 2], [2, 5]]
# number of resources c and c2
c = [6, 8]

# job dependencies/precedences
S = [
    [1, 4],
    [1, 5],
    [2, 9],
    [2, 10],
    [3, 8],
    [4, 6],
    [4, 7],
    [5, 9],
    [5, 10],
    [6, 8],
    [6, 9],
    [7, 8],
]

# Model creation

In [ ]:
pb = ps.SchedulingProblem(name="ResourceConstrainedProject")
# create n jobs
jobs = [ps.FixedDurationTask(name="Job_%i" % (i + 1), duration=p[i]) for i in range(n)]
# create the resources
C1_Ress = [ps.Worker(name="C1_Ress_%i" % i) for i in range(c[0])]
C2_Ress = [ps.Worker(name="C2_Ress_%i" % i) for i in range(c[1])]
# job precedences
for index_job_before, index_job_after in S:
    ps.TaskPrecedence(
        task_before=jobs[index_job_before - 1], task_after=jobs[index_job_after - 1]
    )
# assign resources to jobs
i = 0
for u1, u2 in u:
    if u1 != 0:
        jobs[i].add_required_resource(
            ps.SelectWorkers(
                list_of_workers=C1_Ress, nb_workers_to_select=u1, kind="exact"
            )
        )
    if u2 != 0:
        jobs[i].add_required_resource(
            ps.SelectWorkers(
                list_of_workers=C2_Ress, nb_workers_to_select=u2, kind="exact"
            )
        )
    i += 1

# Makespan

In [ ]:
# there may be several schedules, add the following constraint
# to find the solution available from the python-MIP documentation
ps.TaskStartAt(task=jobs[9], value=15)
pb.add_objective_makespan()

# Solve

In [ ]:
solver = ps.SchedulingSolver(problem=pb)
solution = solver.solve()

# Render Gantt
The solution displayed below should the same as the Python-MIP optimum

In [ ]:
solution.render_gantt_matplotlib()